<a href="https://colab.research.google.com/github/hsw1805/ML-DL/blob/main/05_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블
## 랜덤 포레스트
정형 데이터에 매우 적합

데이터 뽑는 방법
- 부트스트랩 샘플
  - ex) 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다 가정
    
    1.1개의 샘플을 먼저 뽑고 다시 넣음(복원 추출)

  - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듦
  - 1000개의 샘플이 들어있는 가방에서 중복하여 1000개의 샘플을 뽑음



노드 분할하는 방법
- 전체 특성의 개수의 제곱근만큼의 특성을 선택
  - ex) 9개의 특성이 있다면 3개의 특성만 랜덤 선택

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련

분류일 때 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음



In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
#n_jobs = 코어 개수 몇개 쓸지
rf = RandomForestClassifier(n_jobs=-1, random_state = 42)
#교차 검증
score = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(score['train_score']),np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809
